In [4]:
import numpy as np

# Data pre-processing

### In general, before running diffnets your directory structure should look something like this... you have a directory that contains simulation data. In that directory you have a directory for each variant that contains all simulations for that given variant. Then, you should have pdb files to go with the trajectories.

In [12]:
practice_dir
|
|
 ----->simulation_data
         |----->var1
               ------sim1.xtc
               ------sim2.xtc
               ------sim3.xtc
         |----->var2
               ------sim1.xtc
               ------sim2.xtc
               ------sim3.xtc
          |var1.pdb
          |var2.pdb

### Here, copy the diffnets **diffnets/tests/data** to a new directory to practice running diffnets. This dataset only contains two variants and they each have one trajectory so your setup should look something like this:


In [ ]:
practice_dir
|
|
 ----->data
         |----->traj1
               ------beta-peptide1.xtc
         |----->var2
               ------beta-peptide2.xtc
         |beta-peptide1.pdb
         |beta-peptide2.pdb

Now, cd into your practice dir

In [6]:
cd practice_dir/

/Users/mickdub/bowman_lab/scratch/practice_dir


### Now, we need to preprocess this simulation data to turn it into DiffNet input
-- Ultimately, we want to do that through the command line interface (CLI) which looks like this...

**python /path/to/diffnets/diffnets/cli/main.py process {sim_dirs} {pdb_fns} {outdir}**

In [7]:
sim_dirs = np.array(["./data/traj1/",
            "./data/traj2/"])
np.save("./traj_dirs.npy",sim_dirs)

pdb_fns = np.array(["./data/beta-peptide1.pdb",
                    "./data/beta-peptide2.pdb"])
np.save("./pdb_fns.npy",pdb_fns)

It's important to note that the order of **sim_dirs** and **pdb_fns** matters and the pdb order has to correspond to the trajectory order.

### Now, our directory looks something like this: 

In [ ]:
practice_dir
  |
  |pdb_fns.npy
  |traj_dirs.npy
  |
   ----->data
         |----->traj1
               ------beta-peptide1.xtc
         |----->var2
               ------beta-peptide2.xtc
         |beta-peptide1.pdb
         |beta-peptide2.pdb

We are ready to run the data processing step! You should run this next step at the command line (outside of this notebook). Submit this to a CPU node on a cluster and request as many cores as are available on that node.

**python /path/to/diffnets/diffnets/cli/main.py process ./traj_dirs.npy ./pdb_fns.npy ./whitened_data**

### Now, your directory should look something like this:

In [ ]:
practice_dir
  |pdb_fns.npy
  |traj_dirs.npy
  |
   ----->data
           ----->traj1
                   |beta-peptide1.xtc
           ----->var2
                   |beta-peptide2.xtc
           |beta-peptide1.pdb
           |beta-peptide2.pdb
           |
   ----->whitened_data
           ----->indicators
                   |000000.npy
                   |000001.npy
           ----->data
                   |5001 .pt files (one for each simulation frame)
           ----->aligned_xtcs
                   |000000.xtc
                   |000001.xtc
           ----->whitened_xtcs
           |master.pdb
           |traj_dict.pkl
           |traj_lens.npy
           |cm.npy
           |c00.npy
           |uwm.npy
           |wm.npy

We have added a "whitened_data" directory that contains the following:

--an **aligned_xtcs** dir: one .xtc file for each trajectory where the trajectory has been stripped to (C,CA,CB,N) atoms and aligned to the first pdb supplied in pdb_fns (beta-peptide1.pdb). To change what atom selection you want to use, you can optionally supply {atom_sel} to the CLI command that generated this data. See the preprocess_data function in cli/main.py for more details.

--an **indicators** dir: one .npy file for each trajectory where the numbering matches the numbering in aligned_xtcs. The file is a numpy array that has as many values as simulation frames and indicates which variant the trajectory is of. It is zero-indexed so in our case, a value of 0 indicates var1 and a value of 1 indicates var2.

--a **data** dir: one .pt file (pytorch tensor) for each simulation frame. Lots of data means lots of files here.

--master.pdb: The stripped pdb (C,CA,CB,N) containing the set of atoms that is common to all variants. 

--traj_dict.pkl: Helper for plotting later (see Analysis.assign_labels_to_variants in analysis.py for more info).

--cm.npy: center of mass calculated from ALL simulation frames.

--c00.npy: covariance matrix calculated from ALL simulation frames. 

--wm.npy: whitening matrix which will be used as a layer in the DiffNet.

--uwm.npy: unwhitening matrix which will be used as a layer in the DiffNet.

# Training

### Here, we will train the diffnet

Ultimately, we want to run this command:
    
**python /path/to/diffnets/diffnets/cli/main.py train config.yml**

take a look at an example config.yml file at docs/train_sample.yml, but I will also display it here.

Remember, we are still cd'd into practice_dir

In [ ]:
data_dir: './whitened_data'
n_epochs: 10 #How many times do we want to go through the training data?
act_map: [0,1] #initial label for var1 is 0 and for var2 its 1. 
               #Important note, if we had 4 variants (var1, var2, var3, var4) and we wanted to distingusih
               #var1,var2 from var3, var4 act_map would look like [0,0,1,1].
               #The order follows the order we originally put in pdb_fns.npy
lr: 0.0001
n_latent: 50 #Number of dimensions to reduce the data down to.
hidden_layer_sizes: [] #Leave blank array [] for default of 1 hidden layer
                       #that has 4x less nodes than input.
em_bounds: [[0.1,0.4],[0.6,0.9]] #one bound for each variant -- these are reasonable default values
do_em: True
em_batch_size: 150
nntype: 'nnutils.sae' #one of several architectures that can be used, the other
                      #common choice would be nnutils.split_sae.
                      #nnutils.sae classifies based on the entire protein structure
                      #nnutils.split_sae can classify (find differences) based on a specific region
                      #of the structure. For nnutils.split_sae you will have to supply indices via the
                      #close_inds_fn option described below.
batch_size: 32
batch_output_freq: 500
epoch_output_freq: 2
test_batch_size: 1000
frac_test: 0.1
subsample: 1  #You can subsample the data during training. Recommended if training on huge datasets. 
outdir: 'sae_e10_lr0001_lat50_rep0_em' #training will create an output directory. Name it however you like
                                       #just make sure a dir with the same name doesn't already exist
data_in_mem: False  #For small datasets, you can load all the data into memory at once
#close_inds_fn: close_inds.npy #Only necessary if using a split autoencoder. np.array of the atom indices that go into classification task. See train_sample.txt.
#label_spreading: 'gaussian' #Optional parameter to draw initial labels from a normal distribution

### Now your directory should look something like this -- for the sake of brevity the contents of the newly added dir are not exactly as you might see them

In [ ]:
practice_dir
  |pdb_fns.npy
  |traj_dirs.npy
  |
   ----->data
           ----->traj1
                   |beta-peptide1.xtc
           ----->var2
                   |beta-peptide2.xtc
           |beta-peptide1.pdb
           |beta-peptide2.pdb
           |
   ----->whitened_data
           ----->indicators
                   |000000.npy
                   |000001.npy
           ----->data
                   |5001 .pt files (one for each simulation frame)
           ----->aligned_xtcs
                   |000000.xtc
                   |000001.xtc
           ----->whitened_xtcs
           |master.pdb
           |traj_dict.pkl
           |traj_lens.npy
           |cm.npy
           |c00.npy
           |uwm.npy
           |wm.npy
   ----->sae_e10_lr0001_lat50_rep0_em
           |nn_best_polish.pkl + many other pkl files of intermediate networks saved
           |tmp_targets.npy
           |training_loss.npy 

In your new dir **sae_e10_lr0001_lat50_rep0_em** you have trained diffnets (e.g. nn_best_polish.pkl) and some other data that is not a major concern to explore right now. Just make sure you have nn_best_polish.pkl.

# Analysis

### Here, we can simply run this command line command:

**python /path/to/diffnets/diffnets/cli/main.py analyze ./whitened_data ./sae_e10_lr0001_lat50_rep0_em**

### Now, your directory structure will look like this

In [ ]:
practice_dir
  |pdb_fns.npy
  |traj_dirs.npy
  |
   ----->data
           ----->traj1
                   |beta-peptide1.xtc
           ----->var2
                   |beta-peptide2.xtc
           |beta-peptide1.pdb
           |beta-peptide2.pdb
           |
   ----->whitened_data
           ----->indicators
                   |000000.npy
                   |000001.npy
           ----->data
                   |5001 .pt files (one for each simulation frame)
           ----->aligned_xtcs
                   |000000.xtc
                   |000001.xtc
           ----->whitened_xtcs
           |master.pdb
           |traj_dict.pkl
           |traj_lens.npy
           |cm.npy
           |c00.npy
           |uwm.npy
           |wm.npy
   ----->sae_e10_lr0001_lat50_rep0_em
           ----->encodings
                   |000000.npy
                   |000001.npy
           ----->labels
                   |000000.npy
                   |000001.npy
           ----->recon_trajs
                   |000000.xtc
                   |000001.xtc
           ----->morph_label
                   |morph_0-1.pdb
           ----->cluster_1000
                   |clusters.pkl
           |rmsd.npy
           |res-corr100.pml
           |nn_best_polish.pkl + many other pkl files of intermediate networks saved
           |tmp_targets.npy
           |training_loss.npy 

**recon_trajs**: each trajectory gets reconstructed by the DiffNets and an rmsd to the original trajectories is calculated for every 100th frame (**rmsd.npy**)

**labels**: each trajectory has a .npy file where the numpy array contains a DiffNet classification label for each frame

**encodings**: each trajectory has a .npy file where the numpy array contains the latent vector for each frame (i.e. your reduced dimensionality representation)

**morph_label**: A pdb containing 10 representative states morphing from a DiffNet classification label of 0 to 1

**cluster_1000**: A k-centers/k-medoids hybrid clustering using the DiffNet latent space

**res-corr100.pml**: Load whitened_data/master.pdb into pymol, then load this pml file in to get an image like Figure 7 in the paper.